In [1]:
import pandas as pd
import json

/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
with open ("time_unit_meme.json", "r") as f:
    posts_data = json.load(f)

In [5]:
with open("meme_table_combined_text.csv", "r") as f:
    meme_table = pd.read_csv(f)
meme_table.head()

,meme_id,cluster,created_utc,title,selftext,author,num_comments,score,extracted_text,comments_text,created_date,combined_text
0,cawlo8,cluster_4,1.562651e+09,"I have an MIDI keyboard, just no plug for it ):",NaN,YTMusicXD,0,3,When you want to easily make midi; limited pow...,NaN,2019-07-09,"i have an midi keyboard, just no plug for it )..."
1,d72m1n,cluster_4,1.569021e+09,R.I.P,NaN,masnargom,0,95,When Imina room full of antivax children and f...,NaN,2019-09-20,"r.i.p, when imina room full of antivax childre..."
2,bo6k2s,cluster_4,1.557769e+09,It's like a life hack,NaN,emanuelegay,2,4,Connect one Power Bank to another one Unlimite...,This is just plug in a power strip to itself\n...,2019-05-13,"it's like a life hack, connect one power bank ..."
3,cr40bc,cluster_4,1.565948e+09,I Finally Have Unlimited Power,NaN,Zaxoosh,0,64,When you finally get enough karma to post on r...,NaN,2019-08-16,"i finally have unlimited power, when you final..."
4,apoel0,cluster_4,1.549939e+09,Crrevt,NaN,memeboiiii7059,0,7,whwn yoy make a meme wirh one hand Unlimited p...,NaN,2019-02-12,"crrevt, whwn yoy make a meme wirh one hand unl..."


In [6]:
all_posts_df = pd.DataFrame(list(posts_data.items()), columns=['month', 'total_posts'])
all_posts_df['month'] = all_posts_df['month'].astype(int)

In [7]:
meme_table['month'] = pd.to_datetime(meme_table['created_date']).dt.to_period('M').apply(lambda x: x - pd.Period('2008-12', 'M')).apply(lambda x: x.n)

In [8]:
# Count occurrences of each cluster per month
cluster_month_counts = meme_table.groupby(['cluster', 'month']).size().reset_index(name='cluster_count')

In [9]:
# Merge with total posts per month
relative_frequency_df = cluster_month_counts.merge(all_posts_df, on='month', how='left')

In [10]:
# Calculate relative frequency
relative_frequency_df['relative_frequency'] = relative_frequency_df['cluster_count'] / relative_frequency_df['total_posts']

# Replace NaN with 0 for months with no posts
relative_frequency_df['relative_frequency'] = relative_frequency_df['relative_frequency'].fillna(0)

In [11]:
df = relative_frequency_df[['cluster', 'month', 'relative_frequency']]

In [12]:
df.head()

,cluster,month,relative_frequency
0,Acluster_0,120,0.000326
1,Acluster_0,123,0.000303
2,Acluster_0,124,0.000236
3,Acluster_0,125,0.000339
4,Acluster_0,126,0.000204


In [14]:
# content diversity in each time interval for cluster 81
diversity_81 = pd.read_csv("cluster_81_diversity.csv")

In [15]:
diversity_81.head()

,Month,Content Diversity
0,2018-02,0.000000
1,2018-03,0.708929
2,2018-04,0.625534
3,2018-05,0.620715
4,2018-06,0.837571


In [17]:
cluster = df[df['cluster'] == "cluster_81"]
cluster.head()

,cluster,month,relative_frequency
15342,cluster_81,110,0.001612
15343,cluster_81,111,0.002662
15344,cluster_81,112,0.000866
15345,cluster_81,113,0.000795
15346,cluster_81,114,0.000846


In [18]:
diversity_81['Month'] = pd.to_datetime(diversity_81['Month'])

# Calculate the month number (0 for 2008-12, 181 for 2023-12)
base_month = pd.Period('2008-12', freq='M')
diversity_81['Month Number'] = diversity_81['Month'].dt.to_period('M').apply(lambda x: x - base_month).apply(lambda x: x.n)

In [19]:
diversity_81.head()

,Month,Content Diversity,Month Number
0,2018-02-01,0.000000,110
1,2018-03-01,0.708929,111
2,2018-04-01,0.625534,112
3,2018-05-01,0.620715,113
4,2018-06-01,0.837571,114


In [20]:
new_df = cluster.merge(diversity_81, left_on='month', right_on='Month Number', how='left')
new_df.head()

,cluster,month,relative_frequency,Month,Content Diversity,Month Number
0,cluster_81,110,0.001612,2018-02-01,0.000000,110
1,cluster_81,111,0.002662,2018-03-01,0.708929,111
2,cluster_81,112,0.000866,2018-04-01,0.625534,112
3,cluster_81,113,0.000795,2018-05-01,0.620715,113
4,cluster_81,114,0.000846,2018-06-01,0.837571,114


In [22]:
new_df = new_df[['Month', 'relative_frequency', 'Content Diversity']]
new_df.head()

,Month,relative_frequency,Content Diversity
0,2018-02-01,0.001612,0.000000
1,2018-03-01,0.002662,0.708929
2,2018-04-01,0.000866,0.625534
3,2018-05-01,0.000795,0.620715
4,2018-06-01,0.000846,0.837571


In [27]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# new_df['time'] = pd.to_datetime(new_df['Month'])
# new_df.set_index('time', inplace=True)

# Fit ARIMAX model
model = SARIMAX(new_df['relative_frequency'], exog=new_df[['Content Diversity']], order=(1, 0, 0))
result = model.fit()

# Print results
print(result.summary())

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -6.37100D+00    |proj g|=  5.95531D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      3     50      2     0     0   2.776D+01  -6.385D+00
  F =  -6.3848184537845469     

ABNORMAL_TERMINATION_IN_LNSRCH                              
                               SARIMAX Results                                
Dep. Variable:     relative_frequency   No. Observations:                   69
Model:               SARIMAX(1, 0, 0

/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.

 Bad direction in the line search;
   refresh the lbfgs memory and restart the iteration.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/Users/yingrongmao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: Conver

In [28]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:     relative_frequency   No. Observations:                   69
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 440.552
Date:                Fri, 06 Dec 2024   AIC                           -875.105
Time:                        02:12:28   BIC                           -868.403
Sample:                             0   HQIC                          -872.446
                                 - 69                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Content Diversity     0.0016      0.000     12.998      0.000       0.001       0.002
ar.L1                 0.2339      0.079      2.963      0.003       0.079       0.389
sigma2             1.639e-07   1.86e-08      8.801      0.000    1.27e-07       2e-07
===================================================================================
Ljung-Box (L1) (Q):                   0.05   Jarque-Bera (JB):                30.48
Prob(Q):                              0.83   Prob(JB):                         0.00
Heteroskedasticity (H):               0.28   Skew:                             1.06
Prob(H) (two-sided):                  0.00   Kurtosis:                         5.48
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""